In [33]:
import requests
from urllib import parse
from lxml import etree
import random
import time
import math
import pandas as pd
import json
from bs4 import BeautifulSoup
import re 
from requests.packages.urllib3.exceptions import InsecureRequestWarning 
# 禁用安全请求警告 
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

'''
获取拉勾网在线的所有城市名称
页面解析：在源代码可以直接查看到，故为静态网页，可以直接提取，利用BeatuifulSoup解析返回结果
keyword:所要查询关键字，如：'数据分析'
'''
def get_citys(keyword):
    keyword = parse.quote(keyword)
    link = "https://www.lagou.com/jobs/allCity.html?keyword="+keyword+"&px=default&city=%E5%85%A8%E5%9B%BD&positionNum=500+&companyNum=0&isCompanySelected=false&labelWords="
    headers = {
            'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Mobile Safari/537.36'
            }
    r = requests.get(link,headers=headers)
    citys = []
    soup = BeautifulSoup(r.text,'html.parser')
    city_word_list = soup.find_all('ul',class_ ='city_list')
    for i in range(len(city_word_list)):
        city_list = city_word_list[i].find_all('li')
        for j in range(len(city_list)):
            city = city_list[j].a.text.strip()
            citys.append(city)
#             print(city)
    return citys
        

In [34]:
def get_url(city,keyword,gm):
    
    '''
    编码为URL格式,直接传送的地址非URL格式需要利用urllib.parse进行处理
    错误类型：UnicodeEncodeError
     'latin-1' codec can't encode characters in position 87-89: ordinal not in range(256)
    '''
    city = parse.quote(city)
    keyword = parse.quote(keyword)
    gm = parse.quote(gm)
    
    url_parse = "https://www.lagou.com/jobs/positionAjax.json?px=default&city="+city+"&needAddtionalResult=false&gm="+gm
    url_start = "https://www.lagou.com/jobs/list_"+keyword+"?px=default&gm="+ gm + "&city="+city
    return url_parse,url_start

In [35]:
#随机选择执行的IP,防止被封
def get_proxies():#goole免费代理IP替换或者添加，越多越好
    proxie = [
        "134.249.156.3:82",
        "1.198.72.239:9999",
        "103.26.245.190:43328"]
    
    proxies = {"http":str(random.sample(proxie,1))}
    return proxies
def get_agents():#替换你自己的User-Agent,直接运行不了
    agents = [
        '**Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3626.121 Safari/537.36',
        '**Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3729.157 Mobile Safari/537.36'
    ]
    agent = random.sample(agents,1)
    return agent
#定制headers,设置模拟浏览器，反爬
def get_headers(url_start,agent):
    url_start = url_start
    agent = agent
    headers = {
         #所有数据均来自Network-XHR-headers-Requsts Headers
        'Accept': 'application/json, text/javascript, */*; q=0.01',        
        'Referer': url_start,
        #因为本地的User-Agent已经被封锁，此时有了网上的一个虚拟的，为了反爬可以随机使用几个，类似于下面的proxies
#         'Cookie':'user_trace_token=20190515230141-12934d87-ec36-47a4-b6f5-b9fdf6116988; _ga=GA1.2.1677629794.1557932504; _gat=1; LGSID=20190515230142-597afe8c-7722-11e9-99a5-525400f775ce; PRE_UTM=m_cf_cpt_sogou_ztch6; PRE_HOST=; PRE_SITE=; PRE_LAND=https%3A%2F%2Fwww.lagou.com%2Flanding-page%2Fpc%2Fposition2.html%3Futm_source%3Dm_cf_cpt_sogou_ztch6; LGUID=20190515230142-597b0038-7722-11e9-99a5-525400f775ce; Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1557932504; sensorsdata2015jssdkcross=%7B%22distinct_id%22%3A%2216abc03944948-07d289ba2ecf5-353166-921600-16abc039450303%22%2C%22%24device_id%22%3A%2216abc03944948-07d289ba2ecf5-353166-921600-16abc039450303%22%7D; sajssdk_2015_cross_new_user=1; _gid=GA1.2.694067267.1557932513; LG_LOGIN_USER_ID=90be76343c3af8869dd6d12563545c28b420c207df6a38cceeb53f0a5d64da34; LG_HAS_LOGIN=1; _putrc=A7C9A275E2B1F6EB123F89F2B170EADC; JSESSIONID=ABAAABAAADEAAFIAD4122C7F738CEEAE9C99959A9756EB3; login=true; unick=%E5%BC%A0%E5%B9%B3; showExpriedIndex=1; showExpriedCompanyHome=1; showExpriedMyPublish=1; hasDeliver=1; gate_login_token=450a1f502d12ad1d9f320d846f06d1ad92603529c81fa8ff8921a53bcf54a82d; index_location_city=%E5%8C%97%E4%BA%AC; X_HTTP_TOKEN=c7dc7114524f2f9f15523975516bfb19cb3eab323b; Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1557932554; LGRID=20190515230231-76ee1774-7722-11e9-99a5-525400f775ce; SEARCH_ID=caaf8eb6597d4ecbbe9a2d807d68e78f',
        'Host':'www.lagou.com',
        'Origin': 'https://www.lagou.com',
        'User-Agent':  str(agent),
        'X-Anit-Forge-Code': '0',
        'X-Anit-Forge-Token': 'None',
        'X-Requested-With': 'XMLHttpRequest',
        'Connection':'keep_alive'
    }  
    return headers

In [36]:
def get_json(city,keyword,gm,pn):
    #kd是搜索的关键字，若改为“数据分析”即是数据分析相关的数据
    data = { 'first': 'true', 'pn': pn, 'kd': keyword }#keyword若是中文，需要为编码的形式
    url_parse,url_start = get_url(city,keyword,gm)
    agent = get_agents()
    headers = get_headers(url_start,agent)
    proxies = get_proxies()
    print(proxies)
    # 请求首页获取cookies 
    s = requests.Session() 
    s = s.get(url_start, headers=headers, proxies=proxies,timeout=3)   # 请求首页获取cookies ,allow_redirects=False,verify=False
    cookie = s.cookies  # 为此类别获取的cookies 
    response = requests.post(url_parse,headers=headers,data=data,proxies=proxies,cookies=cookie,timeout=5)
    sleep_time = random.randint(4,5)+random.random()
    time.sleep(sleep_time)
    response.encoding = response.apparent_encoding 
#     print(response.text)
    text = json.loads(response.text)
    return text

In [37]:
'''
获取每个城市的全部数据
'''
def get_city_Pages(city,keyword):
    
    data = { 'first': 'true', 'pn': 1, 'kd': keyword }#keyword若是中文，需要为编码的形式
    city = parse.quote(city)
    keyword = parse.quote(keyword)
    url_parse = "https://www.lagou.com/jobs/positionAjax.json?px=default&city="+city+"&needAddtionalResult=false"
    url_start = "https://www.lagou.com/jobs/list_"+keyword+"?px=default&city="+city
    agent = get_agents()
    headers = get_headers(url_start,agent)
    proxies = get_proxies()
     # 请求首页获取cookies 
    s = requests.Session() 
    s = s.get(url_start, headers=headers, proxies=proxies,timeout=3)   # 请求首页获取cookies 
    cookie = s.cookies  # 为此类别获取的cookies 
    response = requests.post(url_parse,headers=headers,data=data,proxies=proxies,cookies=cookie,timeout=5)
    sleep_time = random.randint(4,5)+random.random()
    time.sleep(sleep_time)
    response.encoding = response.apparent_encoding 
#     print(response.text)
    text = json.loads(response.text)
#     print(text)
    totalcount = 0
    if 'content' in text:
        totalcount = text['content']['positionResult']['totalCount']
    else:
        print("Not exits")
    totalPages = math.ceil(float(totalcount)/15)
    city_un = parse.unquote(city)
    if totalPages > 0 :
        print("%s共%s条,可分为%s页"%(city_un,totalcount,totalPages))
#         sleep_time = random.randint(5,15)+random.random()
#         time.sleep(sleep_time)#keyerror问题的解决
#         print("停留%s"%sleep_time)
    else:
#         city_un = parse.quote(city)
        print("%s-----无数据"%(city_un))
#     print("共%s条,可分为%s页"%(totalcount,totalPages))
    return totalPages

In [38]:
def get_totalPages(city,keyword,gm):
    city = city
    keyword = keyword
    totalcount = 0
#     pn = 1
    text = get_json(city,keyword,gm,'1')
    if 'content' in text:
        totalcount = text['content']['positionResult']['totalCount']
    else:
        print("Not exits")
    totalPages = math.ceil(float(totalcount)/15)
    if totalPages > 0 :
        print("%s公司规模为%s共%s条,可分为%s页"%(city,gm,totalcount,totalPages))        
#         sleep_time = random.randint(5,15)+random.random()
#         time.sleep(sleep_time)#keyerror问题的解决
#         print("停留%s"%sleep_time)
    else:
        print("%s公司规模为%s---无数据"%(city,gm))
      
    return totalPages

In [39]:
def get_info(city,keyword,gm,totalPages):
    city = city
    keyword = keyword
    totalPages = totalPages
    position_info_all = []#存储数据
    for page in range(1,totalPages+1):#从1开始循环因为首页的页数编码是1
        time_start = time.time()
        text = get_json(city,keyword,gm,page)
#         print(text)
#         print(proxies)
        if 'content'in text:
            info = text["content"]["positionResult"]["result"] 
            for i in info:
                position_info_single = []
                position_info_single.append(i.get('positionId','NA'))#职位编号
                position_info_single.append(i.get('positionName','NA'))#职位名称
                position_info_single.append(i.get('salary','NA'))#薪酬
                position_info_single.append(i.get('workYear','NA'))#工作经验
                position_info_single.append(i.get('skillLables','NA'))#技能要求
                position_info_single.append(i.get('positionAdvantage','NA'))#职位优势
                position_info_single.append(i.get('education','NA'))#学历要求
                position_info_single.append(i.get('jobNature','NA'))#工作性质
                position_info_single.append(i.get('createTime','NA'))#发布时间
                position_info_single.append(i.get('companyFullName','NA'))#公司
                position_info_single.append(i.get('city','NA'))#城市
                position_info_single.append(i.get('companySize','NA'))#公司规模
                position_info_single.append(i.get('district','NA'))#区域
                position_info_single.append(i.get('financeStage','NA'))#融资情况
                position_info_single.append(i.get('firstType','NA'))#公司类别
                position_info_single.append(i.get('industryField','NA'))#涉及领域
                position_info_single.append(i.get('isSchoolJob','NA'))#是否校招
                position_info_single.append(i.get('subwayline','NA'))#地铁
                position_info_single.append(i.get('stationname','NA'))#站点
                position_info_single.append(i.get('latitude','NA'))#经度
                position_info_single.append(i.get('longitude','NA'))#纬度
                position_info_all.append(position_info_single) 
            print("第%s页爬取成功,position_info_all现有数据%s行"%(str(page),str(len(position_info_all))))
        else:
            print("Not exits")
        sleep_time = random.randint(5,15)+random.random()
        time.sleep(sleep_time)#keyerror问题的解决
        time_end = time.time()
        on_time = time_end-time_start
        print("本页爬行时间%s"%str(on_time))
    return position_info_all

In [52]:
def main():
    #公司规模
    gms = ['2000人以上','少于15人,15-50人','50-150人','150-500人','500-2000人']    
    keyword = '数据分析'
    citys = get_citys(keyword)
    position_info_all = []
    columns = ['职位编号','职位名称','薪酬','工作经验','技能要求','职位优势','学历要求','工作形式','发布时间',
               '公司','城市','规模','区域','融资情况','公司类别','涉及领域','是否校招',
              '地铁','站点','经度','纬度']
    print(len(citys))
    for i in range(0,len(citys)):
        position_info_city = []
        city = citys[i]
        cityPages = get_city_Pages(city,keyword)
        if cityPages > 0 :
            for j in range(len(gms)):
                gm = gms[j]
                totalPages = get_totalPages(city,keyword,gm)
                if totalPages>0:
                    
                    position_info_city_gm = get_info(city,keyword,gm,totalPages)
                    for m in range(len(position_info_city_gm)):
                        item = position_info_city_gm[m]
                        position_info_city.append(item)
                        position_info_all.append(item)
    #                 print(position_info_all)
                    print("%s的公司规模%s的数据爬完"%(city,gm))
                else:
    #                 print("%s的公司规模%s无数据"%(city,gm))
                    sleep_time = random.randint(5,15)+random.random()
                    time.sleep(sleep_time)#keyerror问题的解决
                    print("停留%s"%sleep_time)
            print("position_info_all有%s条数据"%len(position_info_all))
            print("%s的全部数据爬取成功"%city)
            sleep_time = random.randint(5,15)+random.random()
            time.sleep(sleep_time)#keyerror问题的解决
            print("停留%s"%sleep_time)
            df_city = pd.DataFrame(data = position_info_city,columns =columns )
            df_city.to_csv('%s.csv'%(city),index = False,encoding="utf_8_sig")
        else:
            sleep_time = random.randint(5,15)+random.random()
            time.sleep(sleep_time)#keyerror问题的解决
            print("停留%s"%sleep_time)
#         position_info_all.append(position_info_city)
    df_all = pd.DataFrame(data=position_info_all,columns=columns)
    df_all.to_csv('all_city.csv',index = False,encoding="utf_8_sig")
    print("全部数据存储成功（CSV格式）----持续奔跑中！") 
    print(position_info_all)

if __name__ == '__main__':
    main()

293
珠海共3条,可分为1页
{'http': "['218.64.69.79:8080']"}
珠海公司规模为2000人以上共3条,可分为1页
{'http': "['163.125.68.135:8888 ']"}
第1页爬取成功,position_info_all现有数据3行
本页爬行时间16.876307725906372
珠海的公司规模2000人以上的数据爬完
{'http': "['14.115.104.97:808 ']"}
珠海公司规模为少于15人,15-50人---无数据
停留6.7492073387582385
{'http': "['134.249.156.3:82']"}
珠海公司规模为50-150人---无数据
停留11.043163736858903
{'http': "['1.198.72.193:9999']"}
Not exits
珠海公司规模为150-500人---无数据
停留12.172892347874113
{'http': "['183.129.207.86:11206 ']"}
珠海公司规模为500-2000人---无数据
停留10.925358249150833
position_info_city有3条数据
珠海的全部数据爬取成功
停留13.576861105003818
中山共1条,可分为1页
{'http': "['115.28.148.192:8118']"}
中山公司规模为2000人以上---无数据
停留9.102117120811412
{'http': "['134.249.156.3:82']"}
中山公司规模为少于15人,15-50人---无数据
停留7.852819219941637
{'http': "['1.192.243.166:9999 ']"}
中山公司规模为50-150人---无数据
停留15.585111278379154
{'http': "['163.125.68.135:8888 ']"}
中山公司规模为150-500人共1条,可分为1页
{'http': "['171.80.2.110:9999']"}
第1页爬取成功,position_info_all现有数据1行
本页爬行时间20.245324850082397
中山的公司规模150-500人的数据爬完
{'http': 